In [1]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tqdm import tqdm
from encoders.apply_bpe import BPE

In [2]:
#from dataset import *
from Transformer import *
from dataset import *

In [3]:
vocab_size = 1046+2
embedding_size = 64
d_k = 8
words = 64
total_lines = 100
codec_file = open("datasets/english_codec.txt", encoding = 'utf-8')

In [4]:
dataset = Dataset("datasets/english_dataset.txt", "datasets/hindi_dataset.txt", total_lines = 100)

Primary Language Tokens: 1042
Secondary Language Tokens: 1046


In [5]:
dataset.create_dataset()

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 33264.37it/s]

Total Lines added: 1516


In [6]:
encoder_input = tf.keras.layers.Input((words))
decoder_input = tf.keras.layers.Input((words))

In [7]:
encoder_embeddings = tf.keras.layers.Embedding(vocab_size + 2, embedding_size)(encoder_input)
decoder_embeddings = tf.keras.layers.Embedding(vocab_size + 2, embedding_size)(decoder_input)

In [8]:
enc_pos = PositionalEmbedding(embedding_size = embedding_size, words = words)(encoder_embeddings)
dec_pos = PositionalEmbedding(embedding_size = embedding_size, words = words)(decoder_embeddings)

(None, 64, 64)
(None, 64, 64)


In [9]:
encoder_outputs1 = Encoder(d_k = d_k, model_embedding = embedding_size)(enc_pos)
decoder_outputs1 = Decoder(input_words = words, model_embedding = embedding_size, d_k = d_k)((dec_pos, encoder_outputs1[0], encoder_outputs1[1]))

encoder_outputs2 = Encoder(d_k = d_k, model_embedding = embedding_size)(encoder_outputs1[1])
decoder_outputs2 = Decoder(input_words = words, model_embedding = embedding_size, d_k = d_k)((decoder_outputs1, encoder_outputs2[0], encoder_outputs2[1]))

encoder_outputs3 = Encoder(d_k = d_k, model_embedding = embedding_size)(encoder_outputs2[1])
decoder_outputs3 = Decoder(input_words = words, model_embedding = embedding_size, d_k = d_k)((decoder_outputs2, encoder_outputs3[0], encoder_outputs3[1]))

encoder_outputs4 = Encoder(d_k = d_k, model_embedding = embedding_size)(encoder_outputs3[1])
decoder_outputs4 = Decoder(input_words = words, model_embedding = embedding_size, d_k = d_k)((decoder_outputs3, encoder_outputs4[0], encoder_outputs4[1]))

encoder_outputs5 = Encoder(d_k = d_k, model_embedding = embedding_size)(encoder_outputs4[1])
decoder_outputs5 = Decoder(input_words = words, model_embedding = embedding_size, d_k = d_k)((decoder_outputs4, encoder_outputs5[0], encoder_outputs5[1]))

encoder_outputs6 = Encoder(d_k = d_k, model_embedding = embedding_size)(encoder_outputs5[1])
decoder_outputs6 = Decoder(input_words = words, model_embedding = embedding_size, d_k = d_k)((decoder_outputs5, encoder_outputs6[0], encoder_outputs6[1]))

encoder_outputs7 = Encoder(d_k = d_k, model_embedding = embedding_size)(encoder_outputs6[1])
decoder_outputs7 = Decoder(input_words = words, model_embedding = embedding_size, d_k = d_k)((decoder_outputs6, encoder_outputs7[0], encoder_outputs7[1]))


(None, 64, 64)
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([None, 64, 64]))
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([None, 64, 64]))
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([None, 64, 64]))
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([None, 64, 64]))
(None, 64, 64)
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([None, 64, 64]))
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([None, 64, 64]))
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([None, 64, 64]))
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([None, 64, 64]))
(None, 64, 64)
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([None, 64, 64]))
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([None, 64, 64]))
(TensorShape([None, 64, 64]), TensorShape([None, 64, 64]), TensorShape([Non

In [10]:
decoder_outputs = tf.keras.layers.Flatten()(decoder_outputs6)
#decoder_outputs = tf.keras.layers.Dense(500)(decoder_outputs)
decoder_outputs = tf.keras.layers.Dense(vocab_size)(decoder_outputs)
decoder_outputs.shape

TensorShape([None, 1048])

In [11]:
output = tf.nn.softmax(decoder_outputs)

In [12]:
model = tf.keras.Model(inputs = [encoder_input, decoder_input], outputs=[output])

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 64, 64)       67200       ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 64, 64)       67200       ['input_2[0][0]']                
                                                                                              

In [18]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.01))

In [15]:
loss_history = []

In [19]:
history = model.fit((dataset.encoder_inputs, dataset.decoder_inputs), dataset.output_vectors, batch_size = 256, epochs =100)
loss_history += history.history['loss']

Epoch 1/100
6/6 [==============================] - 110s 8s/step - loss: 6.8975
Epoch 2/100
6/6 [==============================] - 46s 8s/step - loss: 6.3647
Epoch 3/100
6/6 [==============================] - 46s 8s/step - loss: 6.0722
Epoch 4/100
6/6 [==============================] - 46s 8s/step - loss: 5.8878
Epoch 5/100
6/6 [==============================] - 45s 7s/step - loss: 5.2204
Epoch 6/100
6/6 [==============================] - 47s 8s/step - loss: 4.6624
Epoch 7/100
6/6 [==============================] - 45s 8s/step - loss: 4.3880
Epoch 8/100
6/6 [==============================] - 46s 8s/step - loss: 4.1365
Epoch 9/100
6/6 [==============================] - 46s 8s/step - loss: 3.9937
Epoch 10/100
6/6 [==============================] - 45s 8s/step - loss: 4.0397
Epoch 11/100
6/6 [==============================] - 45s 8s/step - loss: 3.7389
Epoch 12/100
6/6 [==============================] - 45s 8s/step - loss: 3.5491
Epoch 13/100
6/6 [==============================] - 45s 8s/s

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(20, 20))
plt.plot(loss_history)

In [ ]:
key = "I'd like to tell you about one such child"

In [ ]:
bpe_obj = BPE(codes = codec_file)
key = bpe_obj.process_line(key)

encoding = dataset.primary_vectorizer(key).numpy().reshape(1, 64)
print(encoding)
decoding = np.zeros(shape=(1, 64))

for i in tqdm(range(64)):
    decoding[0, i] = tf.argmax(model((decoding, encoding)), -1)

decoding = tf.convert_to_tensor(decoding)
translation = dataset.convert_to_secondary(decoding)
print("\n")
print(translation.replace('@@ ', ''))